In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import copy 
from tqdm import tqdm
import os
import json
import sys 

sys.path.extend(['..'])
from utils import Config
# from model import LSTMRegressor
from data import LSTMCSVDataset
from datetime import datetime

In [2]:
class LSTMRegressor(nn.Module):
    def __init__(self, input_size, output_size):
        super(LSTMRegressor,self).__init__()
        self.input_size = input_size
        self.output_size = output_size

        self.lstm_1 = nn.LSTM(self.input_size, 100, batch_first=True)
        self.fc1 = nn.Linear(1000, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, output_size)
        self.h1 = self.init_hidden(batch_size=Config['batch_size'],device='cuda:0')

    def init_hidden(self, batch_size, device='cpu'):
        return (torch.zeros(1, batch_size , 100).to(device), torch.zeros(1, batch_size , 100).to(device))
    
    def forward(self, x, device='cpu'):
        out, self.h1 = self.lstm_1(x, self.h1)
        print(out.shape)
        out = out.reshape(out.shape[0],-1)
        print(out.shape)
        out = nn.Tanh()(self.bn2(self.fc1(out)))
        out = nn.Tanh()(self.bn3(self.fc2(out)))
        out = nn.Tanh()(self.bn4(self.fc3(out)))
        out = nn.Tanh()(self.fc4(out))
#         print(out.shape)
        return out

In [3]:
dataset = LSTMCSVDataset(root_path='../'+Config['dataset_path'])

In [4]:
dataset[0][0], dataset[0][1]

(tensor([[ 0.9726, -0.9970, -0.0219, -0.0623, -0.9998, -0.0237,  0.2762, -0.2747,
          -0.9950,  0.1136, -0.2030,  0.9797, -0.0073, -0.0710, -0.2577, -0.9945,
          -0.0314,  0.0967,  0.9872],
         [ 0.9725, -0.9970, -0.0214, -0.0626, -0.9998, -0.0245,  0.2751, -0.2734,
          -0.9950,  0.1134, -0.2005,  0.9790, -0.0080, -0.0715, -0.2572, -0.9945,
          -0.0310,  0.0958,  0.9873],
         [ 0.9724, -0.9970, -0.0209, -0.0626, -0.9998, -0.0249,  0.2733, -0.2721,
          -0.9951,  0.1114, -0.1945,  0.9787, -0.0086, -0.0722, -0.2567, -0.9945,
          -0.0305,  0.0953,  0.9874],
         [ 0.9722, -0.9971, -0.0205, -0.0627, -0.9998, -0.0251,  0.2728, -0.2707,
          -0.9951,  0.1075, -0.1895,  0.9788, -0.0093, -0.0728, -0.2560, -0.9945,
          -0.0300,  0.0949,  0.9875],
         [ 0.9720, -0.9971, -0.0200, -0.0627, -0.9998, -0.0256,  0.2729, -0.2696,
          -0.9952,  0.1025, -0.1858,  0.9794, -0.0098, -0.0735, -0.2556, -0.9945,
          -0.0296,  0.0947, 

In [5]:
model = LSTMRegressor(input_size=19, output_size=7)

In [6]:
model.load_state_dict(torch.load(r'C:\Users\CSCI-538-HP-Z240-lll\Desktop\UnityProjects\XR-Interaction-Toolkit-Examples\ML\models\lstm_alldata_run12_euler\checkpoints\model_999.pth'))

<All keys matched successfully>

In [7]:
model.h1 = model.init_hidden(batch_size=1, device='cpu')

In [8]:
model.forward(dataset[0][0].unsqueeze(0))

torch.Size([1, 10, 100])
torch.Size([1, 1000])


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 512])

In [ ]:
def train(dataloader, dataset_sizes, model, criterion, optimizer, device, num_epochs, batch_size):
    for epoch in range(10):
        model.train()
        for inputs, labels in dataloader:
            with torch.

In [ ]:
w